In [1]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
# import torch.nn as nn
# import torch.optim as optim
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.multiprocessing
torch.multiprocessing.set_start_method('spawn', force=True)
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import random
from tqdm import tqdm
import time
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from pycocotools.coco import COCO

In [2]:
class ConcreteCrackDataset(Dataset):
    def __init__(self, root_dir, split, transforms=None, augment=False):
        
        self.root_dir = root_dir
        self.split = split
        self.transforms = transforms
        self.augment = augment 
        self.split_dir = os.path.join(root_dir, split)
        self.imgs = []

        for root, _, files in os.walk(self.split_dir):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    self.imgs.append(os.path.join(root, file))

        print(f"Found {len(self.imgs)} images in {split}")

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        img_dir, img_name = os.path.split(img_path)
        annot_path = os.path.join(
            img_dir,
            img_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')
        )

        img = Image.open(img_path).convert("RGB")
        width, height = img.size
        boxes = []
        labels = []

        if os.path.exists(annot_path):
            with open(annot_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    cls_id = int(parts[0])
                    x_center = float(parts[1]) * width
                    y_center = float(parts[2]) * height
                    box_width = float(parts[3]) * width
                    box_height = float(parts[4]) * height

                    x1 = max(0, x_center - box_width / 2)
                    y1 = max(0, y_center - box_height / 2)
                    x2 = min(width, x_center + box_width / 2)
                    y2 = min(height, y_center + box_height / 2)

                    if x2 > x1 and y2 > y1:
                        boxes.append([x1, y1, x2, y2])
                        labels.append(1)

        # Optional: Apply horizontal flip before ToTensor
        if self.augment and random.random() > 0.5:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)

            flipped_boxes = []
            for box in boxes:
                x1, y1, x2, y2 = box
                new_x1 = width - x2
                new_x2 = width - x1
                flipped_boxes.append([new_x1, y1, new_x2, y2])
            boxes = flipped_boxes

        # Convert to tensor
        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)

        # Target dict
        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([idx]),
            "area": (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) if len(boxes) > 0 else torch.zeros((0,), dtype=torch.float32),
            "iscrowd": torch.zeros((len(boxes),), dtype=torch.int64),
        }

        # Apply transforms (like ToTensor, Normalize, etc.)
        if self.transforms:
            img, target = self.transforms(img,target)

        return img, target


In [3]:
# Helper function for data augmentation and preprocessing
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class ToTensor:
    def __call__(self, image, target):
        image = T.ToTensor()(image)  # Convert PIL image to tensor
        return image, target

class RandomHorizontalFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            image = F.hflip(image)  # Flip image horizontally

            # Flip bounding boxes
            if target["boxes"].shape[0] > 0:
                width = image.shape[2]  # (C, H, W)
                boxes = target["boxes"]
                boxes[:, [0, 2]] = width - boxes[:, [2, 0]]
                target["boxes"] = boxes

        return image, target

class Normalize:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image, target):
        image = T.Normalize(mean=self.mean, std=self.std)(image)
        return image, target


In [4]:
# Function to get the Faster R-CNN model
def get_faster_rcnn_model(num_classes=2):  # 2classes: background +crack
    # Load a pre-trained model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT')
    
    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

In [5]:
# Function to evaluate the model on validation set
def evaluate(model, data_loader, device):
    model.eval()
    
    # Initialize evaluation metric
    metric = MeanAveragePrecision()
    
    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluating"):
            # Move images to device
            images = list(image.to(device) for image in images)
            
            # Get predictions
            outputs = model(images)
            
            # Convert outputs and targets to format expected by metric
            for i, (output, target) in enumerate(zip(outputs, targets)):
                pred = {
                    'boxes': output['boxes'].cpu(),
                    'scores': output['scores'].cpu(),
                    'labels': output['labels'].cpu()
                }
                
                gt = {
                    'boxes': target['boxes'].cpu(),
                    'labels': target['labels'].cpu()
                }
                
                # Update metric
                metric.update([pred], [gt])
    
    # Compute metric
    result = metric.compute()
    return result


In [6]:
# Function to train one epoch
def train_one_epoch(model, data_loader, optimizer, device):
    model.train()
    
    total_loss = 0
    num_batches = 0 

    # for batch in tqdm(data_loader, desc="Training"):
    for batch in data_loader:

        images, targets = batch  # unpack tuple
        
        # Ensure images are moved to device
        images = [img.to(device) for img in images]
        
        # Move targets to device
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Filter out images with no boxes
        valid_images, valid_targets = [], []
        for img, tgt in zip(images, targets):
            if tgt["boxes"].numel() > 0:
                valid_images.append(img)
                valid_targets.append(tgt)
        
        if len(valid_targets) == 0:
            continue  # skip if no annotations

        optimizer.zero_grad()
        loss_dict = model(valid_images, valid_targets)
        losses = sum(loss for loss in loss_dict.values())

        losses.backward()
        optimizer.step()

        total_loss += losses.item()
        num_batches += 1

    if num_batches == 0:
        return 0.0

    return total_loss / num_batches


In [7]:
# Function to train the model
def train_model(model, train_loader, val_loader, optimizer, scheduler, device, num_epochs):
    best_map = 0.0
    best_model_wts = None
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)
        
        # Train for one epoch
        epoch_loss = train_one_epoch(model, train_loader, optimizer, device)
        print(f"Training Loss: {epoch_loss:.4f}")
        
        # Update learning rate
        scheduler.step()
        
        # Evaluate on validation set
        val_metrics = evaluate(model, val_loader, device)
        val_map = val_metrics['map'].item()
        print(f"Validation mAP: {val_map:.4f}")
        
        # Save best model
        if val_map > best_map:
            best_map = val_map
            best_model_wts = model.state_dict().copy()
            torch.save(best_model_wts, f'best_model_epoch_{epoch+1}.pth')
            print(f"Saved best model with mAP: {best_map:.4f}")
        
        print()
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_map


In [8]:
# Function to visualize predictions
def visualize_prediction(model, img_path, device, threshold=0.5):
    # Set model to evaluation mode
    model.eval()
    
    # Load and transform the image
    img = Image.open(img_path).convert("RGB")
    transform = transforms.Compose([transforms.ToTensor()])
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    # Get prediction
    with torch.no_grad():
        prediction = model(img_tensor)
    
    # Convert image back to numpy for visualization
    img_np = np.array(img)
    
    # Draw bounding boxes on the image
    for idx, box in enumerate(prediction[0]['boxes']):
        score = prediction[0]['scores'][idx].item()
        if score > threshold:
            x1, y1, x2, y2 = box.cpu().numpy().astype(np.int32)
            cv2.rectangle(img_np, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"Crack: {score:.2f}"
            cv2.putText(img_np, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(img_np)
    plt.axis('off')
    plt.show()
    
    return img_np

In [9]:
# Function to evaluate on test set
def evaluate_test_set(model, test_loader, device, output_dir='test_results'):
    model.eval()
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize evaluation metric
    metric = MeanAveragePrecision()
    
    with torch.no_grad():
        for i, (images, targets) in enumerate(tqdm(test_loader, desc="Testing")):
            # Move images to device
            images = list(image.to(device) for image in images)
            
            # Get predictions
            outputs = model(images)
            
            # Convert outputs and targets for metric calculation
            for j, (output, target) in enumerate(zip(outputs, targets)):
                pred = {
                    'boxes': output['boxes'].cpu(),
                    'scores': output['scores'].cpu(),
                    'labels': output['labels'].cpu()
                }
                
                gt = {
                    'boxes': target['boxes'].cpu(),
                    'labels': target['labels'].cpu()
                }
                
                # Update metric
                metric.update([pred], [gt])
            
            # Save visualizations for the first few samples
            if i < 10:  # Save first 10 test examples
                img_np = np.array(images[0].cpu().permute(1, 2, 0))
                img_np = (img_np * 255).astype(np.uint8)
                
                # Draw ground truth boxes in blue
                for box, label in zip(targets[0]['boxes'], targets[0]['labels']):
                    if label == 1:  # Only draw crack boxes
                        x1, y1, x2, y2 = box.cpu().numpy().astype(np.int32)
                        cv2.rectangle(img_np, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue for ground truth
                
                # Draw predicted boxes in green
                for box, score, label in zip(outputs[0]['boxes'], outputs[0]['scores'], outputs[0]['labels']):
                    if score > 0.5 and label == 1:  # Only draw high confidence crack predictions
                        x1, y1, x2, y2 = box.cpu().numpy().astype(np.int32)
                        cv2.rectangle(img_np, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green for predictions
                        cv2.putText(img_np, f"{score:.2f}", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                
                # Save image
                cv2.imwrite(os.path.join(output_dir, f"test_result_{i}.jpg"), cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR))
    
    # Compute final metrics
    result = metric.compute()
    print("Test Results:")
    for k, v in result.items():
        print(f"{k}: {v}")
    
    # Save metrics to file
    with open(os.path.join(output_dir, "test_metrics.txt"), "w") as f:
        for k, v in result.items():
            f.write(f"{k}: {v}\n")
    
    return result


In [10]:
# Define the collate function outside of main()
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# Main function to run the training and evaluation
def main():
        
    # Set random seeds for reproducibility
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
    
    # Set device
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(f"Using device: {device}")
    
    # Data transforms
    train_transform = Compose([
        ToTensor(),
        RandomHorizontalFlip(0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_test_transform = Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    root_dir = '../artifact_folder'

    # Create datasets
    train_dataset = ConcreteCrackDataset(
        root_dir=root_dir,
        split='train/images',
        transforms=train_transform,
        augment=True
    )
    val_dataset = ConcreteCrackDataset(
        root_dir=root_dir,
        split='val/images',
        transforms=val_test_transform,
        augment=True
    )
    test_dataset = ConcreteCrackDataset(
        root_dir=root_dir,
        split='test/images',  
        transforms=val_test_transform,
        augment=True
    )
        
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=4, 
        shuffle=True,
        num_workers=0,
        collate_fn=collate_fn 
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=2,
        shuffle=False,
        num_workers=0,
        collate_fn=collate_fn
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0,
        collate_fn=collate_fn
    )
    
    # Get model
    model = get_faster_rcnn_model(num_classes=2)
    model.to(device)
    
    # Define optimizer and learning rate scheduler
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    
    # Use learning rate scheduler to reduce lr by 0.1 every 3 epochs
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    
    # Train model
    print("Starting training...")
    start_time = time.time()
    model, best_map = train_model(
        model, 
        train_loader, 
        val_loader, 
        optimizer, 
        scheduler, 
        device, 
        num_epochs=15
    )
    end_time = time.time()
    print(f"Training completed in {(end_time - start_time) / 60:.2f} minutes")
    print(f"Best validation mAP: {best_map:.4f}")
    
    # Save final model
    torch.save(model.state_dict(), 'final_faster_rcnn_concrete_crack_detector.pth')
    
    # Evaluate on test set
    print("Evaluating on test set...")
    test_results = evaluate_test_set(model, test_loader, device)
    
    print("Training and evaluation completed!")


In [ ]:
if __name__ == "__main__":
    main()

Using device: cpu
Found 16968 images in train/images
Found 8413 images in val/images
Found 8415 images in test/images
Starting training...
Epoch 1/15
----------


/Users/kwinyarutpoungsangthanakul/Desktop/university/year3/MLOps/Structural-Defects-Network-MLOps/env/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Training Loss: 0.0000


Evaluating: 100%|██████████| 4207/4207 [2:38:24<00:00,  2.26s/it]  


Validation mAP: -1.0000

Epoch 2/15
----------
Training Loss: 0.0000


Evaluating:  80%|███████▉  | 3350/4207 [2:08:41<34:13,  2.40s/it]  